In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

votes_file = '../assets/datasets/votes.csv'
airport_file = '../assets/datasets/Airport_operations.csv'

# Case #1: Congressional Voting Data

After you've downloaded the data from the repository, go ahead and load it with Pandas

In [2]:
votes = pd.read_csv(votes_file)

In [3]:
votes.head(2)

,Unnamed: 0,Class,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,1,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,2,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN



Next, let's define the x and y variables: 

In [4]:
cols = []

for i in range(1,17):
    cols.append('V'+str(i) )

In [8]:
X = votes[cols]
# X.head()

Y = votes['Class']
Y = [1 if x == 'democrat' else 0 for x in votes]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [6]:
X.replace(to_replace=['y','n'], value=[1,0], inplace=True)

/Users/sebozek/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [12]:
# Because dropping NaNs would lead to loss of about half of the data want to impute

In [19]:
# Fill in missing values with random 1 or 0:
for c in X.columns:
    # Make a mask of rows for column where value is NaN
    mask = X[c].isnull()
    # for rows at these columns put in random 1 or 0 size of missing values
    X.loc[mask, c] = np.random.choice(2, size=np.sum(mask))
    
X.isnull().sum()
X.replace(to_replace=['y','n'], value=[1,0], inplace=True)

/Users/sebozek/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sebozek/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,0,1,0,1,1,1,0,0,0,1,1,1,1,1,0,1
1,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
2,1,1,1,0,1,1,0,0,0,0,1,0,1,1,0,0
3,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1
4,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1


Next, create the covariance matrix from the standardized x-values and decompose these values to find the eigenvalues and eigenvectors

In [21]:
Xn = (X-X.mean())/X.std()

In [23]:
eigenvalues, eigenvectors = np.linalg.eig(Xn.corr())

Now, let's check the eigenvalues: 

In [24]:
print eigenvalues

[ 7.11147704  1.42621562  1.11096615  0.89056613  0.82553013  0.73965501
  0.15237021  0.62656712  0.23877907  0.25727441  0.32917842  0.37466906
  0.42673871  0.48263109  0.47136656  0.53601527]


And the eigenvectors: 

In [25]:
print eigenvectors[0:2]

[[-0.19331433 -0.219214   -0.07284893  0.49397615  0.28077477 -0.60465543
  -0.04477858  0.22502587 -0.00516624 -0.05737547 -0.08429077 -0.18112718
  -0.11205055 -0.31852914 -0.1042674  -0.00724679]
 [ 0.0570857  -0.58266796  0.18090615  0.4601253   0.00208551  0.53625481
   0.01260726  0.05862587  0.10441468  0.04112943  0.0764462  -0.04868557
   0.12954084  0.10349669 -0.2682329  -0.03711856]]


To find the principal componants, find the eigenpairs, and sort them from highest to lowest. 

In [27]:
eigen_pairs = [[eigenvalues[i], eigenvectors[:, i]] for i in range(len(eigenvalues))]

eigen_pairs.sort(reverse=True)

In [28]:
[e[0] for e in eigen_pairs]

[7.1114770384824597,
 1.4262156211929689,
 1.1109661490679259,
 0.8905661279907281,
 0.82553012966265005,
 0.73965500922820926,
 0.62656712492159006,
 0.53601526886085782,
 0.48263109408591937,
 0.47136656118581804,
 0.42673870940769187,
 0.37466905565515629,
 0.32917842354294474,
 0.25727440746001939,
 0.23877907149078101,
 0.15237020776429394]

In [54]:
weight_projection = np.hstack((
    weight[0][1].reshape(eigenvectors.shape[1],1),
    weight[1][1].reshape(eigenvectors.shape[1],1)
    ))

In [55]:
print "Weight 2d PCA matrix: \n", weight_projection

Weight 2d PCA matrix: 
[[-0.18803934  0.19357121]
 [ 0.05231849  0.5860793 ]
 [-0.29660826  0.08013044]
 [ 0.31510691 -0.1422561 ]
 [ 0.33401889  0.0385968 ]
 [ 0.26716267  0.07972083]
 [-0.29182075 -0.19643052]
 [-0.32681104 -0.0725546 ]
 [-0.29287877 -0.18058669]
 [ 0.01359228 -0.37071451]
 [-0.07112483  0.4880378 ]
 [ 0.28929338 -0.15955659]
 [ 0.27553751  0.06114016]
 [ 0.28769832 -0.14238328]
 [-0.24744034  0.0173265 ]
 [-0.07138452 -0.28757258]]


Now, calculate the explained variance. Recall the methods we learned in lesson 2.2!

Now, calculate the explained variance and the Cumulative explained variance

In [32]:
for x in range(len(eigenvalues)):
    print "Explained Variance For", str(x+1), ':', eigenvalues[x]/np.sum(eigenvalues) *100, '%'

Explained Variance For 1 : 44.4467314905 %
Explained Variance For 2 : 8.91384763246 %
Explained Variance For 3 : 6.94353843167 %
Explained Variance For 4 : 5.56603829994 %
Explained Variance For 5 : 5.15956331039 %
Explained Variance For 6 : 4.62284380768 %
Explained Variance For 7 : 0.952313798527 %
Explained Variance For 8 : 3.91604453076 %
Explained Variance For 9 : 1.49236919682 %
Explained Variance For 10 : 1.60796504663 %
Explained Variance For 11 : 2.05736514714 %
Explained Variance For 12 : 2.34168159784 %
Explained Variance For 13 : 2.6671169338 %
Explained Variance For 14 : 3.01644433804 %
Explained Variance For 15 : 2.94604100741 %
Explained Variance For 16 : 3.35009543038 %


Now, conduct a PCA using scikit learn

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [33]:
pca = PCA(n_components=3)

X_r = pca.fit(Xn).transform(Xn)

In [34]:
pca.explained_variance_ratio_

array([ 0.44446731,  0.08913848,  0.06943538])

# Case #2: Airport Delays

In [7]:
air = pd.read_csv(airport_file)

In [8]:
air.head()

,airport,year,departures for metric computation,arrivals for metric computation,percent on-time gate departures,percent on-time airport departures,percent on-time gate arrivals,average_gate_departure_delay,average_taxi_out_time,average taxi out delay,average airport departure delay,average airborne delay,average taxi in delay,average block delay,average gate arrival delay
0,ABQ,2004,53971,53818,0.8030,0.7809,0.7921,10.38,9.89,2.43,12.10,2.46,0.83,2.55,10.87
1,ABQ,2005,51829,51877,0.8140,0.7922,0.8001,9.60,9.79,2.29,11.20,2.26,0.89,2.34,10.24
2,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
3,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.40,12.95,2.19,1.29,3.06,12.71
4,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48


First, let's define the x and y variables: Airport is going to be our "Y" variable

In [71]:
Y = air.airport
X = air.ix[:, 1:]

In [93]:
targets = Y.unique()

Then, standardize the x variable for analysis

In [72]:
Xn = (X - X.mean()) / X.std()

Next, create the covariance matrix from the standardized x-values and decompose these values to find the eigenvalues and eigenvectors

In [73]:
eigenvalues_air, eigenvectors_air = np.linalg.eig(Xn.corr())

Then, check your eigenvalues and eigenvectors:

In [77]:
print eigenvalues_air
print '\n'
print eigenvectors_air

[  6.44582241e+00   4.30439944e+00   1.21680022e+00   7.36998015e-01
   5.10347002e-01   3.48937819e-01   1.66600941e-01   1.20077566e-01
   7.31166379e-02   6.41161089e-02   9.57934189e-03   2.33986110e-03
   1.64168946e-04   7.00472649e-04]


[[ -5.74761448e-02   5.25255722e-02   8.08909757e-01  -3.89143377e-01
   -1.51724877e-02  -4.04558235e-01  -3.91819608e-02   1.31976220e-02
   -1.43760537e-01   7.77297605e-03   3.10742763e-02   2.93369275e-02
   -2.29607095e-03   1.99299912e-03]
 [  1.16832935e-01  -4.11909290e-01   1.69808554e-01   4.02576136e-01
   -1.22740151e-01  -1.12320737e-01   2.90266529e-01  -6.08203611e-02
   -7.66594323e-02  -3.32546414e-02   2.17851714e-02  -2.05754595e-02
   -7.07995242e-01   3.17127569e-02]
 [  1.16407898e-01  -4.11581912e-01   1.71084289e-01   4.03871066e-01
   -1.25409356e-01  -1.09894006e-01   2.91346645e-01  -6.55288643e-02
   -9.06577905e-02  -3.22166405e-02   3.61597627e-02   2.65374530e-02
    7.03727342e-01  -3.44394613e-02]
 [ -3.14555883

To find the principal componants, find the eigenpairs, and sort them from highest to lowest. 

In [80]:
weight = [[eigenvectors_air[i], eigenvectors_air[:, i]] for i in range(len(eigenvalues_air))]

Next, Calculate the explained variance

In [84]:
for x in range(len(eigenvalues_air)):
    print "Explained Variance For", X.columns[x], ':', eigenvalues_air[x]/np.sum(eigenvalues_air), '\n' 

 Explained Variance For year : 0.460415886415 

Explained Variance For departures for metric computation : 0.307457102817 

Explained Variance For arrivals for metric computation : 0.0869143011972 

Explained Variance For percent on-time gate departures : 0.0526427153571 

Explained Variance For percent on-time airport departures : 0.0364533572683 

Explained Variance For percent on-time gate arrivals : 0.02492412996 

Explained Variance For average_gate_departure_delay : 0.0119000671954 

Explained Variance For average_taxi_out_time : 0.00857696898315 

Explained Variance For average taxi out delay : 0.00522261699008 

Explained Variance For average airport departure delay : 0.004579722061 

Explained Variance For average airborne delay : 0.000684238706369 

Explained Variance For average taxi in delay : 0.0001671329354 

Explained Variance For average block delay : 1.17263532648e-05 

Explained Variance For average gate arrival delay : 5.00337606696e-05 



Finally, conduct the PCA - use the results about to guide your selection of "n" componants

In [85]:
pca = PCA(n_components=2)

X_r = pca.fit(Xn).transform(Xn)

In [87]:
pca.explained_variance_

array([ 6.43815794,  4.29928125])